In [25]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool, create_react_agent
import datetime
from langchain_community.tools import TavilySearchResults
from langchain import hub
import operator
from typing import Annotated, TypedDict, Union, List
from langchain_core.agents import AgentAction, AgentFinish

In [2]:
load_dotenv()

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [5]:
searchTool=TavilySearchResults(search_depth="basic")

In [6]:
@tool
def getSystemTime(format:str="%Y-%m-%d %H:%M:%S"):
    """
        Returns the current date and time in the specified format
    """
    currentTime=datetime.datetime.now()
    formatted_time=currentTime.strftime(format)
    return formatted_time

In [7]:
tools=[searchTool,getSystemTime]

In [12]:
react_prompt=hub.pull(owner_repo_commit="hwchase17/react")
print(react_prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [14]:
react_agent_runnable=create_react_agent(llm=llmGemini,tools=tools,prompt=react_prompt)

In [20]:
class AgentState(TypedDict):
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[ List[ tuple[ AgentAction, str ] ]  , operator.add  ]

In [22]:
def reason_node(state:AgentState):
    agent_outcome=react_agent_runnable.invoke(input=state)
    return {"agent_outcome":agent_outcome}

In [28]:
def act_node(state:AgentState):
    """
        Will Execute the Tool
    """
    agent_action=state["agent_outcome"]

    # Extract tool name and input from Agent Action 
    tool_name=agent_action.tool
    tool_input=agent_action.tool_input

    # Find the matching tool function
    tool_function=None
    for tool in tools:
        if tool.name==tool_name:
            tool_function=tool
            break

    # Execute the tool with the input
    if tool_function:
        if isinstance(tool_input,dict):
            output=tool_function.invoke(**tool_input)
        else:
            output=tool_function.invoke(input=tool_input)
    else:
        output=f"Tool: `{tool_name}` not found"
            
    return { "intermediate_steps":[ ( agent_action,str(output) ) ] }  # because the operator.add was defined